In [233]:
import numpy as np
from copy import deepcopy
import dionysus as d
import scipy

In [288]:
def SchurComp(M, ind):
    ind = [i if i >=0 else M.shape[0] + i for i in ind]
    nind = [i for i in range(M.shape[0]) if i not in ind]
    A = M[nind, :][:, nind]
    B = M[nind, :][:, ind]
    C = M[ind, :][:, nind]
    D = M[ind, :][:, ind]
    return A - B @ np.linalg.pinv(D) @ C

def combinatorial_Laplacian(Bqplus1, Bq):
    upLaplacian = Bqplus1@Bqplus1.T
    if Bq == 0:
        return upLaplacian
    else:
        downLaplacian = Bq.T@Bq
        return upLaplacian + downLaplacian

def persistent_Laplacian(Bqplus1: np.array, Bq: np.array) -> np.array:
    """
    Inclusion of K in L. Assume the boundaries are ordered the same and first the boundaries in K appear in L.

    Bqplus1: (q+1)-boundary matrix of L.
    Bq: q-boundary matrix of K. (nqK if q=0)
    """
    if type(Bq) == int:
        nqK = Bq
        downL = 0
    else:
        nqK = Bq.shape[1]
        downL = Bq.T@Bq

    if type(Bqplus1) != int:
        upLaplacian = Bqplus1@Bqplus1.T
        nqL = Bqplus1.shape[0]
        IKL = list(range(nqK, nqL))
        upL = SchurComp(upLaplacian, IKL)
    else:
        upL = 0
        
    return upL + downL


In [276]:

f = d.Filtration()
simplices = [([0], 0), ([1], 1), ([2], 2), ([3], 3), ([1,0], 4), ([0,3], 5), ([1,3], 6), ([2,3], 7), ([0,2], 8), ([0,1,3], 9)]
for vertices, time in simplices:
    f.append(d.Simplex(vertices, time))
f.sort()

In [313]:
for s in f:
    for v in s:
        print(type(v))
        print(v, end=",")
    print()


<class 'int'>
0,
<class 'int'>
1,
<class 'int'>
2,
<class 'int'>
3,
<class 'int'>
0,<class 'int'>
1,
<class 'int'>
0,<class 'int'>
3,
<class 'int'>
1,<class 'int'>
3,
<class 'int'>
2,<class 'int'>
3,
<class 'int'>
0,<class 'int'>
2,
<class 'int'>
0,<class 'int'>
1,<class 'int'>
3,


In [324]:
def compute_boundary_matrices(f: d.Filtration, weight_fun):
    t_old = 0
    relevant_times = [t_old]
    n_simplicies_seen_per_time = [[0]]
    n_simplicies_seen_total = [0]

    for s in f:
        q = s.dimension()
        if s.data != t_old:
            relevant_times.append(s.data)
            n_simplicies_seen_per_time.append(deepcopy(n_simplicies_seen_total))
            t_old = s.data

        if q >= len(n_simplicies_seen_total):
            n_simplicies_seen_total.append(0)
        n_simplicies_seen_total[q] += 1

    relevant_times.append(s.data)
    n_simplicies_seen_per_time.append(deepcopy(n_simplicies_seen_total))
    maxq = len(n_simplicies_seen_per_time[-1])
    for qi in range(len(n_simplicies_seen_per_time)):
        n_simplicies_seen_per_time[qi] = n_simplicies_seen_per_time[qi] + [0]*(maxq-len(n_simplicies_seen_per_time[qi]))

    def simplices_at_time(t):
        i = 0
        if t == np.inf:
            return n_simplicies_seen_per_time[-1]
        while relevant_times[i] <= t:
            i += 1
            if i == len(relevant_times) - 1:
                return n_simplicies_seen_per_time[-1]
        return n_simplicies_seen_per_time[i]

    simplices_at_end = simplices_at_time(np.inf)
    boundary_matrices = [0]+[np.zeros((simplices_at_end[q-1], simplices_at_end[q])) for q in range(1,maxq)]
    name_to_idx = [{} for _ in range(maxq)]
    for s in f:
        q = s.dimension()

        name = str(s).split("<")[1].split(">")[0]
        idx = len(name_to_idx[q])
        name_to_idx[q][name] = idx

        if q > 0:
            for i, bdry_simplex in enumerate(s.boundary()):
                name_bdry_simplex = str(bdry_simplex).split("<")[1].split(">")[0]  
                idx_bdry_simplex = name_to_idx[q-1][name_bdry_simplex]

                # Boundaries as described in OG paper
                # boundary_matrices[q][idx_bdry_simplex, idx] = weight_fun(name)/weight_fun(name_bdry_simplex)*(-1)**i

                # Assuming product weights
                for v in s:
                    if v not in bdry_simplex:
                        name_bdry_simplex = str(v)
                        break
                boundary_matrices[q][idx_bdry_simplex, idx] = weight_fun(name_bdry_simplex)*(-1)**i

    return boundary_matrices, name_to_idx, simplices_at_time

def weight_fun(x):
    if x in ["0", "2"]:
        return 0
    return 1

boundary_matrices, name_to_idx, simplices_at_time = compute_boundary_matrices(f, weight_fun)
boundary_matrices[1]

array([[-1., -1.,  0.,  0.,  0.],
       [ 0.,  0., -1.,  0.,  0.],
       [ 0.,  0.,  0., -1.,  0.],
       [ 0.,  0.,  1.,  0.,  0.]])

In [303]:
# Fails because of machine precision !!!!!!!
simplices_at_time(0)

[1, 0, 0]

In [325]:
import scipy.linalg


def persistent_Laplacian_filtration(q, boundary_matrices, s, t, simplices_at_time):
    if q > 0:
        Bq = boundary_matrices[q][:simplices_at_time(s)[q-1], :simplices_at_time(s)[q]]
    else:
        Bq = simplices_at_time(s)[q]
    
    if q+1 <= len(simplices_at_time(t)):
        Bqplus1 = boundary_matrices[q+1][:simplices_at_time(t)[q], :simplices_at_time(t)[q+1]]
    else:
        Bqplus1 = 0
    return persistent_Laplacian(Bqplus1, Bq)

q = 0

for t in range(1,10):
    for s in range(t):

        Lap = persistent_Laplacian_filtration(q, boundary_matrices, s, t-1, simplices_at_time)
        bijm1 = np.sum(np.linalg.eig(Lap)[0] < 1e-5)

        Lap = persistent_Laplacian_filtration(q, boundary_matrices, s, t, simplices_at_time)
        bij = np.sum(np.linalg.eig(Lap)[0] < 1e-5)

        Lap = persistent_Laplacian_filtration(q, boundary_matrices, s-1, t-1, simplices_at_time)
        bim1jm1 = np.sum(np.linalg.eig(Lap)[0] < 1e-5)

        Lap = persistent_Laplacian_filtration(q, boundary_matrices, s-1, t, simplices_at_time)
        bim1j = np.sum(np.linalg.eig(Lap)[0] < 1e-5)

        if bijm1-bij-(bim1jm1-bim1j) > 0:
            print(s, t)

0 4
3 6
2 7


In [139]:
A = np.arange(6).reshape((3,2))
print(A, A.shape)
B = np.arange(4).reshape((2,2))
print(B, B.shape)
(A@B)

[[0 1]
 [2 3]
 [4 5]] (3, 2)
[[0 1]
 [2 3]] (2, 2)


array([[ 2,  3],
       [ 6, 11],
       [10, 19]])